In [ ]:
import aiohttp
import asyncio
import json
import logging

from IPython.html import widgets
from IPython.display import display as ipydisplay

from utils import colorify_log_handler

In [ ]:
colorify_log_handler(
    logging.getLogger().handlers[0],  # IPython by default inject one
    log_lineno = False,
    time_fmt = '%H:%M:%S'
)

logger = logging.getLogger('bench_rest_api')
logger.setLevel(logging.DEBUG)

logging.getLogger('asyncio').setLevel(logging.DEBUG)

In [ ]:
logger.info('This is info')
logger.debug('我會說中文喔')
logger.error('……人家不是喜歡才跟你講話的喔')
logger.warning('笨蛋')

In [ ]:
!curl -s -XGET "http://localhost:5566/" | python -m json.tool

In [ ]:
!curl -s -XGET "http://localhost:5566/quote/uniform" | python -m json.tool

In [ ]:
%%bash

ab -c 10 -n 10 "http://localhost:5566/quote?slow=true"

## Basic

In [ ]:
@asyncio.coroutine
def quote_simple(url='http://localhost:5566/quote/uniform', slow=False):
    r = yield from aiohttp.request(
        'GET', url, params={'slow': True} if slow else {}
    )
    if r.status != 200:
        logger.error('Unsuccessful response [Status: %s (%d)]' 
                     % (r.reason, r.status))
        r.close(force=True)
        return None
    quote_json = yield from r.json()
    return quote_json['quote']

In [ ]:
loop = asyncio.get_event_loop()

To run a simple asyncio corountine.

In [ ]:
coro = quote_simple()
quote = loop.run_until_complete(coro)
quote

Internally asyncio wraps it with [`asyncio.Task`].
So the following works equivalently.

[`asyncio.Task`]:  https://docs.python.org/3.4/library/asyncio-task.html#asyncio.Task

In [ ]:
task = asyncio.Task(quote_simple())
quote = loop.run_until_complete(task)
quote

However, `coro` is `corountine`, and `task` is `Task` (subclass of [`Future`]).

One can use `asyncio.ensure_future` to make sure having a Future obj returned.

[`Future`]: https://docs.python.org/3.4/library/asyncio-task.html#asyncio.Future

In [ ]:
type(coro), type(task)

Passing wrong URL gives error

In [ ]:
quote = loop.run_until_complete(
    quote_simple(url='http://localhost:5566/quote/uniform?part=100')
)

### Multiple Concurrent Requests

In [ ]:
@asyncio.coroutine
def quote_many_naive(num_quotes=1):
    coroutines = [
        quote_simple(slow=True) for i in range(num_quotes)
    ]
    quotes = yield from (asyncio.gather(*coroutines))
    return quotes

In [ ]:
%%time
quotes = loop.run_until_complete(quote_many_naive(500))

This is not helping since we open 500 connections at a time. It is slower than expected.

### Limiting connection pool size

Ref on [official site](http://aiohttp.readthedocs.org/en/latest/client.html#limiting-connection-pool-size).

In [ ]:
@asyncio.coroutine
def quote(conn, url='http://localhost:5566/quote/uniform', slow=False):
    r = yield from aiohttp.request(
        'GET', url, params={'slow': True} if slow else {},
        connector=conn
    )
    if r.status != 200:
        logger.error('Unsuccessful response [Status: %s (%d)]' 
                     % (r.reason, r.status))
        r.close(force=True)
        return None
    quote_json = yield from r.json()
    r.close(force=True)
    return quote_json['quote']

@asyncio.coroutine
def quote_many(num_quotes=1, conn_limit=20):
    conn = aiohttp.TCPConnector(keepalive_timeout=1, force_close=True, limit=conn_limit)
    coroutines = [
        quote(conn) for i in range(num_quotes)
    ]
    quotes = yield from (asyncio.gather(*coroutines))
    return quotes

In [ ]:
%%time
quotes = loop.run_until_complete(quote_many(2000, conn_limit=50))

In [ ]:
def quote_with_lock(semaphore, url='http://localhost:5566/quote/uniform'):
    with (yield from semaphore):
        r = yield from aiohttp.request('GET', url)
        if r.status != 200:
            logger.error('Unsuccessful response [Status: %s (%d)]' 
                         % (r.reason, r.status))
            r.close(force=True)
            return None
    quote_json = yield from r.json()
    r.close(force=True)
    return quote_json['quote']

@asyncio.coroutine
def quote_many(num_quotes=1, conn_limit=20):
    semaphore = asyncio.Semaphore(conn_limit)
    coroutines = [
        quote_with_lock(semaphore) for i in range(num_quotes)
    ]
    quotes = yield from (asyncio.gather(*coroutines))
    return quotes

In [ ]:
%%time
quotes = loop.run_until_complete(quote_many(2000, conn_limit=50))